<a href="https://colab.research.google.com/github/nickLin1225/Final-Psy-LineBot/blob/main/final_psy_LineBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install line-bot-sdk flask
!pip install pyngrok
!ngrok authtoken 2QrnEFtjjMkbdQs609UXhiHh7JP_41yYQ94u84CsaxFZoiaJ5
!git clone https://github.com/THUDM/ChatGLM2-6B

In [ ]:
# 添加peft、opencc
!pip install -r /content/ChatGLM2-6B/requirements.txt

In [ ]:
!unzip chatGLM2_6B_QLoRA_t16_v5-20230729T023644Z-001.zip

In [ ]:
from transformers import AutoTokenizer, AutoModel, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import torch
base_model_path = "THUDM/chatglm2-6b"
peft_model_path = "/content/chatGLM2_6B_QLoRA_t16_v5/checkpoint-1700"

config = PeftConfig.from_pretrained(peft_model_path)


tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)
base_model = AutoModel.from_pretrained(base_model_path,
                      trust_remote_code=True,
                      device='cuda')

model = PeftModel.from_pretrained(base_model, peft_model_path)

In [ ]:
from flask import Flask, request, abort
from linebot.models.events import FollowEvent
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import *
import requests
from pyngrok import ngrok
import json
import opencc
import csv

# 建置主程序
app = Flask(__name__)
# Channel access token
line_bot_api = LineBotApi("lDOvYWU5DjtPpSVFrNKfNAejGKKSZvKYvhVMBR1ND1zmh8Got+JtoF4EgpWRconwzhxkKWWpwbIsby4+vLETeg+dMhUU8M6mL+7dmNFnHl1QbtdT66wv3gIlj90GKuECO7r/q/wVPkkNnCGtnsfRKAdB04t89/1O/w1cDnyilFU=")
# Channel secret
handler = WebhookHandler("f8f7fe98ab2ebbf5c4af4ace91a7b7f1")


# 啟動server對外接口，使line能夠丟消息進來
@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
'''
回應用戶，歡迎用的文字消息

'''

@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage('哈囉，歡迎來到心理聊天室，有什麼問題都可以跟我說呦！')]
    )

In [ ]:
@handler.add(MessageEvent, message=TextMessage)
def handle_line_text(event):
    response = chat_with_llm(event.source.user_id, event.message.text)
    line_bot_api.reply_message(event.reply_token, messages=[TextSendMessage(response)])

def chat_with_llm(user_id, input_text):
    user_profile = line_bot_api.get_profile(user_id)
    prmpts = "现在你是一名专业的心理咨商师，根据下列发问者所提供之问题一步步进行思考，并以第一人称口吻给出答复。\n问题描述："

    cc = opencc.OpenCC('tw2s')
    converted_input = cc.convert(input_text)

    cc = opencc.OpenCC('s2tw')
    response, history = model.chat(tokenizer, prmpts + converted_input, history=[], repetition_penalty=1.2)
    response = '\n'.join(response.replace(". ", ".").split()).replace(",", "，").replace(".", ". ")
    converted_response = cc.convert(response)
    field_names = ["user_id", "display_name", "picture_url", "status_message", "input_text", "response"]

    # 將用戶資料以及聊天紀錄存至users.csv
    with open("/content/drive/MyDrive/users.csv", "a", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=field_names)

        if csvfile.tell() == 0:
            writer.writeheader()

        writer.writerow({
            "user_id": user_profile.user_id,
            "display_name": user_profile.display_name,
            "picture_url": user_profile.picture_url,
            "status_message": user_profile.status_message,
            "input_text": input_text,
            "response": converted_response
        })
    return converted_response

In [ ]:
# 主程序運行
port = 5000
# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))
app.run()